In [41]:
 pip install MetaTrader5

Note: you may need to restart the kernel to use updated packages.


In [1]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import time
import struct
# import trinket 

In [2]:
# подключимся к MetaTrader 5
if not mt5.initialize():
    print("initialize() failed")
    quit()

In [3]:
#Получим тики EURRUB и USDRUB со вчерашнего дня по сегодняшний день
def take_ticks():
    eurrub_ticks = mt5.copy_ticks_range("GBPUSD",
                                         datetime.now() - timedelta(days=1),
                                         datetime.now(),
                                         mt5.COPY_TICKS_ALL)

    usdrub_ticks = mt5.copy_ticks_range("EURUSD", 
                                         datetime.now() - timedelta(days=1),
                                         datetime.now(),
                                         mt5.COPY_TICKS_ALL)
    return eurrub_ticks, usdrub_ticks

In [4]:
# подготовка запроса
symbol_dol = "EURUSD"
symbol_euro = "GBPUSD"
symbol_dol_info =mt5.symbol_info(symbol_dol)
symbol_euro_info =mt5.symbol_info(symbol_euro)
lot = 0.01 
point_dol = mt5.symbol_info(symbol_dol).point 
price_dol = mt5.symbol_info_tick(symbol_dol).ask
price_euro = mt5.symbol_info_tick(symbol_euro).ask
point_euro = mt5.symbol_info(symbol_euro).point
# deviation = отклонение
deviation = 20 
# запросы

#  struct MqlTradeRequest
#   {
#    ENUM_TRADE_REQUEST_ACTIONS    action;           // Тип выполняемого действия
#    ulong                         magic;            // Штамп эксперта (идентификатор magic number)
#    ulong                         order;            // Тикет ордера
#    string                        symbol;           // Имя торгового инструмента
#    double                        volume;           // Запрашиваемый объем сделки в лотах
#    double                        price;            // Цена 
#    double                        stoplimit;        // Уровень StopLimit ордера
#    double                        sl;               // Уровень Stop Loss ордера
#    double                        tp;               // Уровень Take Profit ордера
#    ulong                         deviation;        // Максимально приемлемое отклонение от запрашиваемой цены
#    ENUM_ORDER_TYPE               type;             // Тип ордера
#    ENUM_ORDER_TYPE_FILLING       type_filling;     // Тип ордера по исполнению
#    ENUM_ORDER_TYPE_TIME          type_time;        // Тип ордера по времени действия
#    datetime                      expiration;       // Срок истечения ордера (для ордеров типа ORDER_TIME_SPECIFIED)
#    string                        comment;          // Комментарий к ордеру
#    ulong                         position;         // Тикет позиции
#    ulong                         position_by;      // Тикет встречной позиции
#   };
   

# покупка доллара
request = { 
"action": mt5.TRADE_ACTION_DEAL, 
"symbol": symbol_dol, 
"volume": lot, 
"type": mt5.ORDER_TYPE_BUY, 
"price": price_dol, 
"stoplimit": price_dol*1.25,
"deviation": deviation, 
"magic": 123456, 
"comment": "купили доллар", 
"type_time": mt5.ORDER_TIME_GTC, 
"type_filling": mt5.ORDER_FILLING_FOK, 
}

# продажа доллара
request_2 = {
"action": mt5.TRADE_ACTION_DEAL,# Совершить сделку прямо сейчас
"symbol": symbol_dol, # Сделка USDRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL, # Покупаем или продаём
#"price": price, # Для .TRADE_ACTION_DEAL не требуется
"deviation": deviation, # Максимальное отклонение от заданной цены
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продали доллар",
"type_time": mt5.ORDER_TIME_GTC,
"type_filling": mt5.ORDER_FILLING_FOK,   #не будет продажи если нет необходимого обьема 
    
                                            #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}

request2 = {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro, # Сделка EURRUB
"volume": lot, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_SELL,
"price": price_euro,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "продажа евро",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,   #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
request2_2= {
"action": mt5.TRADE_ACTION_DEAL,
"symbol": symbol_euro, # Сделка EURRUB
#"volume": 1.0, # Объем который мы покупаем
"type": mt5.ORDER_TYPE_BUY,
#"price": price,
"deviation": deviation,
"magic": 123456, #Идентификатор эксперта. Позволяет организовать аналитическую обработку торговых ордеров.
"comment": "python script open",
"type_time": mt5.ORDER_TIME_GTC,        # Ордер будет находиться в очереди до тех пор, пока не будет снят
"type_filling": mt5.ORDER_FILLING_FOK,      #Означает согласие совершить сделку 
                                            #по максимально доступному на рынке 
                                            #объему в пределах указанного в ордере. 
                                            #В случае невозможности полного исполнения ордер 
                                            #будет исполнен на доступный объем, а неисполненный объем ордера будет отменен.
}
# request-ПОКУПКА ДОЛЛАРА
# request_2-ПРОДАЖА ДОЛЛАРА
# request2-ПРОДАЖА ЕВРО
# request2_2-ПОКУПКА ЕВРО


In [5]:
# Завершим подключение - // завершать подключение будем в самом конце а если программа 
# будет использоваться все время то и откл не будем


In [6]:
def ticks_to_df(eurrub_ticks,usdrub_ticks):
    #Переведем имеющиеся у нас данные в DataFrame
    eurrub_ticks_frame = pd.DataFrame(eurrub_ticks)
    usdrub_ticks_frame = pd.DataFrame(usdrub_ticks)
    #Время в MetaTrader 5 хранится в UTC таймзоне, поэтому переведем в привычный нам вид
    eurrub_ticks_frame['time']=pd.to_datetime(eurrub_ticks_frame['time'], unit='s')
    usdrub_ticks_frame['time']=pd.to_datetime(usdrub_ticks_frame['time'], unit='s')

    #Удалим ненужные столбцы
    eurrub_ticks_frame.drop('ask', axis = 1, inplace = True)
    eurrub_ticks_frame.drop('flags', axis = 1, inplace = True)
    eurrub_ticks_frame.drop('volume_real', axis = 1, inplace = True)
    eurrub_ticks_frame.drop('last', axis = 1, inplace = True)
    eurrub_ticks_frame.drop('volume', axis = 1, inplace = True)

    usdrub_ticks_frame.drop('bid', axis = 1, inplace = True)
    usdrub_ticks_frame.drop('flags', axis = 1, inplace = True)
    usdrub_ticks_frame.drop('volume_real', axis = 1, inplace = True)
    usdrub_ticks_frame.drop('last', axis = 1, inplace = True)
    usdrub_ticks_frame.drop('volume', axis = 1, inplace = True)
    return eurrub_ticks_frame, usdrub_ticks_frame
    #eurrub_ticks_frame['time_msc']=pd.to_datetime(eurrub_ticks_frame['time_msc'], unit='ms')
    #usdrub_ticks_frame['time_msc']=pd.to_datetime(usdrub_ticks_frame['time_msc'], unit='ms')

In [7]:
def change_ticks_df(eurrub_ticks_frame,usdrub_ticks_frame):
    position_euro = 0
    position_usd = 0
    old_euro = eurrub_ticks_frame['bid'][0]
    old_usd = usdrub_ticks_frame['ask'][0]
    current_usd = old_usd
    current_euro = old_euro

    spread = []

    while(position_euro <= len(eurrub_ticks_frame['time'])):

        if(eurrub_ticks_frame['time_msc'][position_euro] >= usdrub_ticks_frame['time_msc'][position_usd]):

            current_usd = usdrub_ticks_frame['ask'][position_usd]
            spread.append(current_euro - current_usd)
            old_usd = usdrub_ticks_frame['ask'][position_usd]
            if position_usd >= len(usdrub_ticks_frame['time']) - 1:
                break
            position_usd += 1
        else:

            current_euro = eurrub_ticks_frame['bid'][position_euro]
            spread.append(current_euro - old_usd)
            if position_euro >= len(eurrub_ticks_frame['time']) - 1:
                break
            position_euro += 1

    return spread

In [19]:
def open_close(spread):
    #максимальное и минимальное значение
    max_spread = max(spread)
    min_spread = min(spread)
    #Найдём медиану
    spread = np.array(spread)
    median = np.median(spread)
    print('median:     ',median,
          '\nmax spread: ', max_spread,
          '\nmin_spread: ',min_spread)
    Open = median + (max_spread - median) * 0.1
    print(Open)
    Close = median + (min_spread - median) * 0.1
    print(Close)
    return Open, Close

In [20]:
def request_open_pos(): 
    result =mt5.order_send(request) 
    result2 = mt5.order_send(request2) 
    print(result)
    print(result2)

In [21]:
def request_close_pos():
    result = mt5.order_send(request_2) 
    result2 = mt5.order_send(request2_2) 
    print(result)
    print(result2)

In [22]:
take_ticks()
a = ticks_to_df(take_ticks()[0],take_ticks()[1])
b = change_ticks_df(a[0], a[1])
Open,Close = open_close(b)


median:      0.16073000000000004 
max spread:  0.16466000000000003 
min_spread:  0.15737
0.16076930000000003
0.16069640000000004


In [26]:
while(True):
    if datetime.now().minute % 20 == 0:
        take_ticks()
        a = ticks_to_df(take_ticks()[0],take_ticks()[1])
        b = change_ticks_df(a[0], a[1])
        Open,Close = open_close(b)
    last_tick_usdrub = mt5.symbol_info_tick("EURUSD").bid
    last_tick_GPBUSD = mt5.symbol_info_tick("GBPUSD").ask
    last_spread = last_tick_GPBUSD - last_tick_usdrub
    if last_spread > Open:
        request_open_pos() # Написать Функции
    elif last_spread < Close:
        print('\n')
        request_close_pos() # Написать Функции



OrderSendResult(retcode=10009, deal=213894387, order=216771336, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=310, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894388, order=216771337, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=311, retcode_external=0, request=

OrderSendResult(retcode=10009, deal=213894398, order=216771347, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=321, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894399, order=216771348, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=322, retcode_external=0, request=Tr

OrderSendResult(retcode=10009, deal=213894409, order=216771358, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=332, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894410, order=216771359, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=333, retcode_external=0, request=Tr

OrderSendResult(retcode=10009, deal=213894423, order=216771372, volume=0.01, price=1.20411, bid=0.0, ask=0.0, comment='Request executed', request_id=344, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894424, order=216771373, volume=0.01, price=1.20411, bid=0.0, ask=0.0, comment='Request executed', request_id=345, retcode_external=0, request=

OrderSendResult(retcode=10009, deal=213894435, order=216771384, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=356, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894436, order=216771385, volume=0.01, price=1.2041, bid=0.0, ask=0.0, comment='Request executed', request_id=357, retcode_external=0, request=Tr

...

OrderSendResult(retcode=10009, deal=213894534, order=216771482, volume=0.01, price=1.20412, bid=0.0, ask=0.0, comment='Request executed', request_id=449, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894535, order=216771483, volume=0.01, price=1.20412, bid=0.0, ask=0.0, comment='Request executed', request_id=450, retcode_external=0, request=

OrderSendResult(retcode=10009, deal=213894545, order=216771493, volume=0.01, price=1.2040899999999999, bid=0.0, ask=0.0, comment='Request executed', request_id=460, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894546, order=216771494, volume=0.01, price=1.2040899999999999, bid=0.0, ask=0.0, comment='Request executed', request_id=461, retcod

OrderSendResult(retcode=10009, deal=213894562, order=216771509, volume=0.01, price=1.20401, bid=0.0, ask=0.0, comment='Request executed', request_id=476, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894563, order=216771510, volume=0.01, price=1.20401, bid=0.0, ask=0.0, comment='Request executed', request_id=477, retcode_external=0, request=

OrderSendResult(retcode=10009, deal=213894578, order=216771526, volume=0.01, price=1.20405, bid=0.0, ask=0.0, comment='Request executed', request_id=491, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='EURUSD', volume=0.01, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=1, type_filling=0, type_time=0, expiration=0, comment='продали доллар', position=0, position_by=0))
OrderSendResult(retcode=10014, deal=0, order=0, volume=0.0, price=0.0, bid=0.0, ask=0.0, comment='Invalid volume', request_id=0, retcode_external=0, request=TradeRequest(action=1, magic=123456, order=0, symbol='GBPUSD', volume=0.0, price=0.0, stoplimit=0.0, sl=0.0, tp=0.0, deviation=20, type=0, type_filling=0, type_time=0, expiration=0, comment='python script open', position=0, position_by=0))


OrderSendResult(retcode=10009, deal=213894579, order=216771527, volume=0.01, price=1.20405, bid=0.0, ask=0.0, comment='Request executed', request_id=492, retcode_external=0, request=

KeyboardInterrupt: 

In [20]:
last_spread

0.16188000000000002

In [22]:
datetime.now().minute

27

In [22]:
Close * 0.99

0.15677937000000006

In [21]:
Open

0.16346600000000006

In [63]:
mt5.shutdown()

True

###  Далее черновой код не использующийся в основной программе

In [24]:

lasttick=mt5.symbol_info_tick("GBPUSD")

lasttick.bid

1.36712

In [8]:
define EXPERT_MAGIC 123456   // MagicNumber эксперта
def OnStart():
   MqlTradeRequest request={0};
   MqlTradeResult  result={0};

   request.action   =TRADE_ACTION_DEAL;                     # тип торговой операции
   request.symbol   ="GBPUSD";                              # символ
   request.volume   =0.1;                                   # объем в 0.1 лот
   request.type     =ORDER_TYPE_BUY;                        # тип ордера
   request.price    =SymbolInfoDouble(Symbol(),SYMBOL_ASK); # цена для открытия
   request.deviation=5;                                     # допустимое отклонение от цены
   request.magic    =EXPERT_MAGIC;                         # MagicNumber ордера

   if(!OrderSend(request,result))
      PrintFormat("OrderSend error %d",GetLastError());     # если отправить запрос не удалось, вывести код ошибки
   PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

SyntaxError: invalid syntax (<ipython-input-8-7a7daa745a3e>, line 1)

In [ ]:
PrintFormat("retcode=%u  deal=%I64u  order=%I64u",result.retcode,result.deal,result.order);

In [ ]:
def user_login():
    try:
    login(
    login,                    // номер счета
    password="PASSWORD",      // пароль
    server="SERVER",          // имя сервера, как оно задано в терминале
    timeout=TIMEOUT           // таймаут
    )
    except  TimeoutErrore

# account=25115284
# authorized=mt5.login(account, password="gqrtz0lbdm")
# user_login()

def errors():
     print("initialize() failed, error code =",mt5.last_error())
#   

In [ ]:
Class Money:
    name = "RUB"
    price
# 
Class Dollar(Money):
    price
# 

Class Euro(Money):
    price
    def __init__(self,price):
        self.price = price
    def __del__():
        print(seld.price,"удален")
# 
euro = Euro()
